In [21]:
import pandas as pd
import numpy as np

In [3]:
product = pd.read_csv('/Users/swimmingcircle/Documents/Suplai/instacart-market-basket-analysis/products.csv')

In [4]:
print(product.shape)
product.head()
#prodcut id is unique

(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [5]:
order_product_prior = pd.read_csv('/Users/swimmingcircle/Documents/Suplai/instacart-market-basket-analysis/order_products__prior.csv')

In [6]:
print(order_product_prior.shape)
order_product_prior.head(20)
# print(order_product_prior['add_to_cart_order'].nunique())

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0
5,2,17794,6,1
6,2,40141,7,1
7,2,1819,8,1
8,2,43668,9,0
9,3,33754,1,1


(94496, 4)

In [9]:
orders = pd.read_csv('/Users/swimmingcircle/Documents/Suplai/instacart-market-basket-analysis/orders.csv')

In [10]:
print(orders.shape)
print(orders['user_id'].nunique())
print(orders['user_id'].max())
print(orders['user_id'].min())
orders.head(10)

#order_id is unique

(3421083, 7)
206209
206209
1


,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [11]:
orders['order_hour_of_day'].value_counts()

10    288418
11    284728
15    283639
14    283042
13    277999
12    272841
16    272553
9     257812
17    228795
18    182912
8     178201
19    140569
20    104292
7      91868
21     78109
22     61468
23     40043
6      30529
0      22758
1      12398
5       9569
2       7539
4       5527
3       5474
Name: order_hour_of_day, dtype: int64

In [12]:
filter_orders = orders[orders['order_id'] <= 10000]
filter_orders['user_id'].nunique()

9529

In [13]:
order_products__train = pd.read_csv('/Users/swimmingcircle/Documents/Suplai/instacart-market-basket-analysis/order_products__train.csv')

In [14]:
print(order_products__train.shape)
order_products__train.head(20)

(1384617, 4)


,order_id,product_id,add_to_cart_order,reordered
0,1,49302,1,1
1,1,11109,2,1
2,1,10246,3,0
3,1,49683,4,0
4,1,43633,5,1
5,1,13176,6,0
6,1,47209,7,0
7,1,22035,8,1
8,36,39612,1,0
9,36,19660,2,1


In [15]:
departments = pd.read_csv('/Users/swimmingcircle/Documents/Suplai/instacart-market-basket-analysis/departments.csv')

In [16]:
print(departments.shape)
departments.head(10)

(21, 2)


,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [17]:
aisles = pd.read_csv('/Users/swimmingcircle/Documents/Suplai/instacart-market-basket-analysis/aisles.csv')

In [18]:
print(aisles.shape)
aisles.head(10)

(134, 2)


,aisle_id,aisle
0,1,prepared soups salads
1,2,specialty cheeses
2,3,energy granola bars
3,4,instant foods
4,5,marinades meat preparation
5,6,other
6,7,packaged meat
7,8,bakery desserts
8,9,pasta sauce
9,10,kitchen supplies


### Merge Data

In [24]:
from functools import partial

# merge train and prior together iteratively, to fit into 8GB kernel RAM
# split df indexes into parts
indexes = np.linspace(0, len(order_product_prior), num=10, dtype=np.int32)

# initialize it with train dataset
order_details = pd.merge(
                left=order_products__train,
                 right=orders, 
                 how='left', 
                 on='order_id'
        ).apply(partial(pd.to_numeric, errors='ignore', downcast='integer'))

# add order hierarchy
order_details = pd.merge(
                left=order_details,
                right=goods[['product_id', 
                             'aisle_id', 
                             'department_id']].apply(partial(pd.to_numeric, 
                                                             errors='ignore', 
                                                             downcast='integer')),
                how='left',
                on='product_id'
)

print(order_details.shape, op_train.shape)

# delete (redundant now) dataframes
del op_train

order_details.head()

NameError: name 'goods' is not defined